In [25]:
from qiskit.aqua.operators import Z, Y, X
from qiskit.aqua.operators import StateFn

QUBITS = 4
operatorZ = Z ^ Z ^ Z ^ Z
operatorX = X ^ X ^ X ^ X
operatorY = Y ^ Y ^ Y ^ Y

def quantum_layer(initial_parameters):
    # expecting parameters to be a numpy array
    quantumRegister = QuantumRegister(QUBITS)
    quantumCircuit = QuantumCircuit(quantumRegister)
    
    quantumCircuit.h(range(4))

    for i in range(len(initial_parameters)):
      quantumCircuit.ry(initial_parameters[i] * np.pi, i)
   
    psi = StateFn(quantumCircuit)
    
    # two ways of doing the same thing
    expectationX = (~psi @ operatorX @ psi).eval()
    expectationZ = psi.adjoint().compose(operatorZ).compose(psi).eval().real
    expectationY = (~psi @ operatorY @ psi).eval()
    
    expectationZ = np.abs(np.real(expectationZ))
    expectations = [expectationX, expectationY, expectationZ, 
                    expectationX + expectationY + expectationZ] 

    return np.array(expectations)

In [31]:
from tensorflow import keras
class Linear(keras.layers.Layer):
    def __init__(self, batch_size=64, units=4,**kwargs):
        super(Linear, self).__init__(**kwargs)

    def get_config(self):
        config = super(Linear, self).get_config()
        return config

    def call(self, inputs):
        final_output = []
        for i in range(inputs.shape[0]):
          pred = quantum_layer(inputs[i].numpy())
          final_output.append(list(pred))
        return tf.convert_to_tensor(final_output)

In [35]:
import sys, os
import tensorflow as tf
sys.path.insert(1, os.path.join(sys.path[0], '..'))

class Mlp(object):
    def __init__(
            self,
            layer_sizes,
            output_size=None,
            activations=None,
            output_activation=None,
            use_bias=True,
            kernel_initializer=None,
            bias_initializer=tf.zeros_initializer(),
            kernel_regularizer=None,
            bias_regularizer=None,
            activity_regularizer=None,
            kernel_constraint=None,
            bias_constraint=None,
            trainable=True,
            name=None,
            name_internal_layers=True
    ):
        """
        Stacks len(layer_sizes) dense layers on top of each other, with an additional layer with output_size neurons,
         if specified.
         """
        self.layers = []
        internal_name = None
        # If object isn't a list, assume it is a single value that will be repeated for all values
        if not isinstance(activations, list):
            activations = [activations for _ in layer_sizes]
        # end if
        # If there is one specifically for the output, add it to the list of layers to be built
        if output_size is not None:
            layer_sizes = layer_sizes + [output_size]
            activations = activations + [output_activation]
        # end if

        new_layer = tf.layers.Dense(
            4,
            activation='softmax',
            use_bias=use_bias,
            kernel_initializer=kernel_initializer,
            bias_initializer=bias_initializer,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
            activity_regularizer=activity_regularizer,
            kernel_constraint=kernel_constraint,
            bias_constraint=bias_constraint,
            trainable=trainable,
            name=internal_name
        )
        layer_2 = Linear(new_layer)
        layer_3 = Linear(layer_2)

        self.layers.append(new_layer)
        self.layers.append(layer_2)
        self.layers.append(layer_3)

    # end __init__

    def __call__(self, inputs, *args, **kwargs):
        outputs = [inputs]
        for layer in self.layers:
            outputs.append(layer(outputs[-1]))
        # end for
        return outputs[-1]
    # end __call__
# end Mlp

In [44]:
class GraphNN(object):
    def __init__(
        self,
        var,
        mat,
        msg,
        loop,
        MLP_depth=3,
        MLP_weight_initializer=tf.keras.initializers.GlorotNormal(),
        MLP_bias_initializer=tf.zeros_initializer,
        RNN_cell=tf.compat.v1.nn.rnn_cell.LSTMCell,
        Cell_activation=tf.nn.relu,
        Msg_activation=tf.nn.relu,
        Msg_last_activation=None,
        float_dtype=tf.float32,
        name='GraphNN'
    ):
        """
        Receives three dictionaries: var, mat and msg.
        ○ var is a dictionary from variable names to embedding sizes.
          That is: an entry var["V1"] = 10 means that the variable "V1" will have an embedding size of 10.

        ○ mat is a dictionary from matrix names to variable pairs.
          That is: an entry mat["M"] = ("V1","V2") means that the matrix "M" can be used to mask messages from "V1" to "V2".

        ○ msg is a dictionary from function names to variable pairs.
          That is: an entry msg["cast"] = ("V1","V2") means that one can apply "cast" to convert messages from "V1" to "V2".

        ○ loop is a dictionary from variable names to lists of dictionaries:
          {
            "mat": the matrix name which will be used,
            "transpose?": if true then the matrix M will be transposed,
            "fun": transfer function (python function built using tensorflow operations,
            "msg": message name,
            "var": variable name
          }
          If "mat" is None, it will be the identity matrix,
          If "transpose?" is None, it will default to false,
          if "fun" is None, no function will be applied,
          If "msg" is false, no message conversion function will be applied,
          If "var" is false, then [1] will be supplied as a surrogate.

          That is: an entry loop["V2"] = [ {"mat":None,"fun":f,"var":"V2"}, {"mat":"M","transpose?":true,"msg":"cast","var":"V1"} ] enforces the following update rule for every timestep:
            V2 ← tf.append( [ f(V2), Mᵀ × cast(V1) ] )
        """
        self.var, self.mat, self.msg, self.loop, self.name = var, mat, msg, loop, name

        self.MLP_depth = MLP_depth
        self.MLP_weight_initializer = MLP_weight_initializer
        self.MLP_bias_initializer = MLP_bias_initializer
        self.RNN_cell = RNN_cell
        self.Cell_activation = Cell_activation
        self.Msg_activation = Msg_activation
        self.Msg_last_activation = Msg_last_activation
        self.float_dtype = float_dtype

        # Check model for inconsistencies
        self.check_model()

        # Initialize the parameters
        with tf.compat.v1.variable_scope(self.name):
            with tf.compat.v1.variable_scope('parameters'):
                self._init_parameters()
            # end parameter scope
        # end GraphNN scope
    # end __init__

    def check_model(self):
        # Procedure to check model for inconsistencies
        for v in self.var:
            if v not in self.loop:
                raise Warning(
                    'Variable {v} is not updated anywhere! Consider removing it from the model'.format(v=v))
            # end if
        # end for

        for v in self.loop:
            if v not in self.var:
                raise Exception(
                    'Updating variable {v}, which has not been declared!'.format(v=v))
            # end if
        # end for

        for mat, (v1, v2) in self.mat.items():
            if v1 not in self.var:
                raise Exception(
                    'Matrix {mat} definition depends on undeclared variable {v}'.format(mat=mat, v=v1))
            # end if
            if v2 not in self.var and type(v2) is not int:
                raise Exception(
                    'Matrix {mat} definition depends on undeclared variable {v}'.format(mat=mat, v=v2))
            # end if
        # end for

        for msg, (v1, v2) in self.msg.items():
            if v1 not in self.var:
                raise Exception(
                    'Message {msg} maps from undeclared variable {v}'.format(msg=msg, v=v1))
            # end if
            if v2 not in self.var:
                raise Exception(
                    'Message {msg} maps to undeclared variable {v}'.format(msg=msg, v=v2))
            # end if
        # end for
    # end check_model

    def _init_parameters(self):
        # Init LSTM cells
        self._RNN_cells = {
            v: self.RNN_cell(
                d,
                activation=self.Cell_activation
            ) for (v, d) in self.var.items()
        }
        # Init message-computing MLPs
        self._msg_MLPs = {
            msg: Mlp(
                layer_sizes=[self.var[vin] for _ in range(self.MLP_depth)],
                output_size=self.var[vout],
                activations=[
                    self.Msg_activation for _ in range(self.MLP_depth)],
                output_activation=self.Msg_last_activation,
                use_bias=True,
                kernel_initializer=self.MLP_weight_initializer(),
                bias_initializer=self.MLP_weight_initializer(),
                name=msg,
                name_internal_layers=True,
                kernel_regularizer=None,
                bias_regularizer=None,
                activity_regularizer=None,
                kernel_constraint=None,
                bias_constraint=None,
                trainable=True,
            ) for msg, (vin, vout) in self.msg.items()
        }
    # end _init_parameters

    def __call__(self, adjacency_matrices, initial_embeddings, time_steps, LSTM_initial_states={}):
        with tf.compat.v1.variable_scope(self.name):
            with tf.compat.v1.variable_scope("assertions"):
                assertions = self.check_run(
                    adjacency_matrices, initial_embeddings, time_steps, LSTM_initial_states)
            # end assertion variable scope
            with tf.control_dependencies(assertions):
                states = {}
                for v, init in initial_embeddings.items():
                    h0 = init
                    c0 = tf.zeros_like(
                        h0, dtype=self.float_dtype) if v not in LSTM_initial_states else LSTM_initial_states[v]
                    states[v] = tf.compat.v1.rnn.LSTMStateTuple(h=h0, c=c0)
                # end

                # Build while loop body function
                def while_body(t, states):
                    new_states = {}
                    for v in self.var:
                        inputs = []
                        for update in self.loop[v]:
                            if 'var' in update:
                                y = states[update['var']].h
                                if 'fun' in update:
                                    y = update['fun'](y)
                                # end if
                                if 'msg' in update:
                                    y = self._msg_MLPs[update['msg']](y)
                                # end if
                                if 'mat' in update:
                                    y = tf.matmul(
                                        adjacency_matrices[update['mat']],
                                        y,
                                        adjoint_a=update['transpose?'] if 'transpose?' in update else False
                                    )
                                # end if
                                inputs.append(y)
                            else:
                                inputs.append(
                                    adjacency_matrices[update['mat']])
                            # end if var in update
                        # end for update in loop
                        inputs = tf.concat(inputs, axis=1)
                        with tf.compat.v1.variable_scope('{v}_cell'.format(v=v)):
                            _, new_states[v] = self._RNN_cells[v](
                                inputs=inputs, state=states[v])
                        # end cell scope
                    # end for v in var
                    return (t+1), new_states
                # end while_body

                _, last_states = tf.while_loop(
                    lambda t, states: tf.less(t, time_steps),
                    while_body,
                    [0, states]
                )
            # end assertions
        # end Graph scope
        return last_states
    # end __call__


In [45]:

import sys, os
import tensorflow as tf

sys.path.insert(1, os.path.join(sys.path[0], '..'))

def build_network(d):

    # Define hyperparameters
    d = d
    learning_rate = 2e-5
    l2norm_scaling = 1e-10
    global_norm_gradient_clipping_ratio = 0.65
    
    tf.compat.v1.disable_eager_execution()

    # Placeholder for answers to the decision problems (one per problem)
    cn_exists = tf.compat.v1.placeholder( tf.float32, shape = (None,), name = 'cn_exists' )
    # Placeholders for the list of number of vertices and edges per instance
    n_vertices  = tf.compat.v1.placeholder( tf.int32, shape = (None,), name = 'n_vertices')
    n_edges     = tf.compat.v1.placeholder( tf.int32, shape = (None,), name = 'n_edges')
    # Placeholder for the adjacency matrix connecting each vertex to its neighbors 
    M_matrix   = tf.compat.v1.placeholder( tf.float32, shape = (None,None), name = "M" )
    # Placeholder for the adjacency matrix connecting each vertex to its candidate colors
    VC_matrix = tf.compat.v1.placeholder( tf.float32, shape = (None,None), name = "VC" )
    # Placeholder for chromatic number (one per problem)
    chrom_number = tf.compat.v1.placeholder( tf.float32, shape = (None,), name = "chrom_number" )
    # Placeholder for the number of timesteps the GNN is to run for
    time_steps  = tf.compat.v1.placeholder( tf.int32, shape = (), name = "time_steps" )
    #Placeholder for initial color embeddings for the given batch
    colors_initial_embeddings = tf.compat.v1.placeholder( tf.float32, shape=(None,d), name= "colors_initial_embeddings")
    
    
    # All vertex embeddings are initialized with the same value, which is a trained parameter learned by the network
    total_n = tf.shape(input=M_matrix)[1]
    v_init = tf.compat.v1.get_variable(initializer=tf.random.normal((1,d)), dtype=tf.float32, name='V_init')
    vertex_initial_embeddings = tf.tile(
        tf.compat.v1.div(v_init, tf.sqrt(tf.cast(d, tf.float32))),
        [total_n, 1]
    )
    
    
    
    # Define GNN dictionary
    GNN = {}

    # Define Graph neural network
    gnn = GraphNN(
        {
            # V is the set of vertex embeddings
            'V': d,
            # C is for color embeddings
            'C': d
        },
        {
            # M is a V×V adjacency matrix connecting each vertex to its neighbors
            'M': ('V','V'),
            # MC is a VxC adjacency matrix connecting each vertex to its candidate colors
            'VC': ('V','C')
        },
        {
            # V_msg_C is a MLP which computes messages from vertex embeddings to color embeddings
            'V_msg_C': ('V','C'),
            # C_msg_V is a MLP which computes messages from color embeddings to vertex embeddings
            'C_msg_V': ('C','V')
        },
        {   # V(t+1) <- Vu( M x V, VC x CmsgV(C) )
            'V': [
                {
                    'mat': 'M',
                    'var': 'V'
                },
                {
                    'mat': 'VC',
                    'var': 'C',
                    'msg': 'C_msg_V'
                }
            ],
            # C(t+1) <- Cu( VC^T x VmsgC(V))
            'C': [
                {
                    'mat': 'VC',
                    'msg': 'V_msg_C',
                    'transpose?': True,
                    'var': 'V'
                }
            ]
        }
        ,
        name='graph-coloring'
    )

    # Populate GNN dictionary
    GNN['gnn']          = gnn
    GNN['cn_exists'] = cn_exists
    GNN['n_vertices']   = n_vertices
    GNN['n_edges']      = n_edges
    GNN["M"]           = M_matrix
    GNN["VC"] = VC_matrix
    GNN["chrom_number"] = chrom_number
    GNN["time_steps"]   = time_steps
    GNN["colors_initial_embeddings"] = colors_initial_embeddings

    # Define V_vote, which will compute one logit for each vertex
    V_vote_MLP = Mlp(
        layer_sizes = [ d for _ in range(3) ],
        activations = [ tf.nn.relu for _ in range(3) ],
        output_size = 1,
        name = 'V_vote',
        kernel_initializer = tf.compat.v1.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"),
        bias_initializer = tf.compat.v1.zeros_initializer()
        )
    
    # Get the last embeddings
    last_states = gnn(
      { "M": M_matrix, "VC": VC_matrix, 'chrom_number': chrom_number },
      { "V": vertex_initial_embeddings, "C": colors_initial_embeddings },
      time_steps = time_steps
    )
    GNN["last_states"] = last_states
    V_n = last_states['V'].h
    C_n = last_states['C'].h
    
    
    # Compute a vote for each embedding
    V_vote = tf.reshape(V_vote_MLP(V_n), [-1])

    # Compute the number of problems in the batch
    num_problems = tf.shape(input=n_vertices)[0]

    # Compute a logit probability for each problem
    pred_logits = tf.while_loop(
        cond=lambda i, pred_logits: tf.less(i, num_problems),
        body=lambda i, pred_logits:
            (
                (i+1),
                pred_logits.write(
                    i,
                    tf.reduce_mean(input_tensor=V_vote[tf.reduce_sum(input_tensor=n_vertices[0:i]):tf.reduce_sum(input_tensor=n_vertices[0:i])+n_vertices[i]])
                )
            ),
        loop_vars=[0, tf.TensorArray(size=num_problems, dtype=tf.float32)]
        )[1].stack()
    # Convert logits into probabilities
    GNN['predictions'] = tf.sigmoid(pred_logits)

    # Compute True Positives, False Positives, True Negatives, False Negatives, accuracy
    GNN['TP'] = tf.reduce_sum(input_tensor=tf.multiply(cn_exists, tf.cast(tf.equal(cn_exists, tf.round(GNN['predictions'])), tf.float32)))
    GNN['FP'] = tf.reduce_sum(input_tensor=tf.multiply(cn_exists, tf.cast(tf.not_equal(cn_exists, tf.round(GNN['predictions'])), tf.float32)))
    GNN['TN'] = tf.reduce_sum(input_tensor=tf.multiply(tf.ones_like(cn_exists)-cn_exists, tf.cast(tf.equal(cn_exists, tf.round(GNN['predictions'])), tf.float32)))
    GNN['FN'] = tf.reduce_sum(input_tensor=tf.multiply(tf.ones_like(cn_exists)-cn_exists, tf.cast(tf.not_equal(cn_exists, tf.round(GNN['predictions'])), tf.float32)))
    GNN['acc'] = tf.reduce_mean(input_tensor=tf.cast(tf.equal(cn_exists, tf.round(GNN['predictions'])), tf.float32))

    # Define loss
    GNN['loss'] = tf.reduce_mean(input_tensor=tf.nn.sigmoid_cross_entropy_with_logits(labels=cn_exists, logits=pred_logits))

    # Define optimizer
    optimizer = tf.compat.v1.train.AdamOptimizer(name='Adam', learning_rate=learning_rate)

    # Compute cost relative to L2 normalization
    vars_cost = tf.add_n([ tf.nn.l2_loss(var) for var in tf.compat.v1.trainable_variables() ])

    # Define gradients and train step
    grads, _ = tf.clip_by_global_norm(tf.gradients(ys=GNN['loss'] + tf.multiply(vars_cost, l2norm_scaling),xs=tf.compat.v1.trainable_variables()),global_norm_gradient_clipping_ratio)
    GNN['train_step'] = optimizer.apply_gradients(zip(grads, tf.compat.v1.trainable_variables()))
    
    GNN['C_n'] = C_n
    
    # Return GNN dictionary
    return GNN
#end


In [46]:
import time, sys, os, random
import numpy as np
import tensorflow as tf

def load_weights(sess, path, scope=None):
    if os.path.exists(path):
        # Restore saved weights
        print("Restoring saved model ... ")
        # Create model saver
        if scope is None:
            saver = tf.train.Saver()
        else:
            saver = tf.train.Saver(var_list=tf.get_collection(
                tf.GraphKeys.GLOBAL_VARIABLES, scope=scope))
        # end
        saver.restore(sess, "%s/model.ckpt" % path)
    else:
        raise Exception('Path does not exist!')
    # end if
# end


def save_weights(sess, path, scope=None):
    # Create /tmp/ directory to save weights
    if not os.path.exists(path):
        os.makedirs(path)
    # end if
    # Create model saver
    if scope is None:
        saver = tf.train.Saver()
    else:
        saver = tf.train.Saver(var_list=tf.get_collection(
            tf.GraphKeys.GLOBAL_VARIABLES, scope=scope))
    # end
    saver.save(sess, "%s/model.ckpt" % path)
    print("MODEL SAVED IN PATH: {path}\n".format(path=path))


In [47]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys, os, time, random, argparse, timeit
import tensorflow as tf
import numpy as np
from itertools import islice
from functools import reduce
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

class InstanceLoader(object):

    def __init__(self,path):
        self.path = path
        self.filenames = [ path + '/' + x for x in os.listdir(path) ]
        random.shuffle(self.filenames)
        self.reset()
    #end

    def get_instances(self, n_instances):
        for i in range(n_instances):
            # Read graph from file
            Ma,chrom_number,diff_edge = read_graph(self.filenames[self.index])
            f = self.filenames[self.index]
            
            Ma1 = Ma
            Ma2 = Ma.copy()

            if diff_edge is not None:
                # Create a (UNSAT/SAT) pair of instances with one edge of difference
                # The second instance has one edge more (diff_edge) which renders it SAT
                Ma2[diff_edge[0],diff_edge[1]] = Ma2[diff_edge[1],diff_edge[0]] =1
            #end

            # Yield both instances
            yield Ma1,chrom_number,f
            yield Ma2,chrom_number,f

            if self.index + 1 < len(self.filenames):
                self.index += 1
            else:
                self.reset()
        #end
    #end

    def create_batch(instances):

        # n_instances: number of instances
        n_instances = len(instances)
        
        # n_vertices[i]: number of vertices in the i-th instance
        n_vertices  = np.array([ x[0].shape[0] for x in instances ])
        # n_edges[i]: number of edges in the i-th instance
        n_edges     = np.array([ len(np.nonzero(x[0])[0]) for x in instances ])
        # n_colors[i]: number of colors in the i-th instance
        n_colors = np.array( [x[1] for x in instances])
        # total_vertices: total number of vertices among all instances
        total_vertices  = sum(n_vertices)
        # total_edges: total number of edges among all instances
        total_edges     = sum(n_edges)
        # total_colors: total number of colors among all instances
        total_colors = sum(n_colors)

        # Compute matrices M, MC
        # M is the adjacency matrix
        M              = np.zeros((total_vertices,total_vertices))
        # MC is a matrix connecting each problem nodes to its colors candidates
        MC = np.zeros((total_vertices, total_colors))        

        # Even index instances are SAT, odd are UNSAT
        cn_exists = np.array([ 1-(i%2) for i in range(n_instances) ])

        for (i,(Ma,chrom_number,f)) in enumerate(instances):
            # Get the number of vertices (n) and edges (m) in this graph
            n, m, c = n_vertices[i], n_edges[i], n_colors[i]
            # Get the number of vertices (n_acc) and edges (m_acc) up until the i-th graph
            n_acc = sum(n_vertices[0:i])
            m_acc = sum(n_edges[0:i])
            c_acc = sum(n_colors[0:i])
            #Populate MC
            MC[n_acc:n_acc+n,c_acc:c_acc+c] = 1

            # Get the list of edges in this graph
            edges = list(zip(np.nonzero(Ma)[0], np.nonzero(Ma)[1]))

            # Populate M
            for e,(x,y) in enumerate(edges):
                if Ma[x,y] == 1:
                  M[n_acc+x,n_acc+y] = M[n_acc+y,n_acc+x] = 1
                #end if
            #end for
        #end for
        return M, n_colors, MC, cn_exists, n_vertices, n_edges, f
    #end

    def get_batches(self, batch_size):
        for i in range( len(self.filenames) // batch_size ):
            instances = list(self.get_instances(batch_size))
            yield InstanceLoader.create_batch(instances)
        #end
    #end
    
    def get_test_batches(self, batch_size, total_instances):
        for i in range( total_instances ):
            instances = list(self.get_instances(batch_size))
            yield InstanceLoader.create_batch(instances)
        #end
    #end

    def reset(self):
        random.shuffle(self.filenames)
        self.index = 0
    #end
#end

def read_graph(filepath):
    with open(filepath,"r") as f:

        line = ''

        # Parse number of vertices
        while 'DIMENSION' not in line: line = f.readline();
        n = int(line.split()[1])
        Ma = np.zeros((n,n),dtype=int)
        
        # Parse edges
        while 'EDGE_DATA_SECTION' not in line: line = f.readline();
        line = f.readline()
        while '-1' not in line:
            i,j = [ int(x) for x in line.split() ]
            Ma[i,j] = 1
            line = f.readline()
        #end while

        # Parse diff edge
        while 'DIFF_EDGE' not in line: line = f.readline();
        diff_edge = [ int(x) for x in f.readline().split() ]

        # Parse target cost
        while 'CHROM_NUMBER' not in line: line = f.readline();
        chrom_number = int(f.readline().strip())

    #end
    return Ma,chrom_number,diff_edge
#end


def run_training_batch(sess, model, batch, batch_i, epoch_i, time_steps, d, verbose=True):
    M, C, VC, cn_exists, n_vertices, n_edges, f = batch
    # Generate colors embeddings
    ncolors = np.sum(C)
    # We define the colors embeddings outside, randomly. They are not learnt by the GNN (that can be improved)
    colors_initial_embeddings = np.random.rand(ncolors, d)

    # Define feed dict
    feed_dict = {
        model['M']: M,
        model['VC']: VC,
        model['chrom_number']: C,
        model['time_steps']: time_steps,
        model['cn_exists']: cn_exists,
        model['n_vertices']: n_vertices,
        model['n_edges']: n_edges,
        model['colors_initial_embeddings']: colors_initial_embeddings
    }

    outputs = [model['train_step'], model['loss'], model['acc'], model['predictions'], model['TP'], model['FP'],
               model['TN'], model['FN']]

    # Run model
    loss, acc, predictions, TP, FP, TN, FN = sess.run(outputs, feed_dict=feed_dict)[-7:]

    if verbose:
        # Print stats
        print(
            '{train_or_test} Epoch {epoch_i} Batch {batch_i}\t|\t(n,m,batch size)=({n},{m},{batch_size})\t|\t(Loss,'
            'Acc)=({loss:.4f},{acc:.4f})\t|\tAvg. (Sat,Prediction)=({avg_sat:.4f},{avg_pred:.4f})'.format(
                train_or_test='Train',
                epoch_i=epoch_i,
                batch_i=batch_i,
                loss=loss,
                acc=acc,
                n=np.sum(n_vertices),
                m=np.sum(n_edges),
                batch_size=n_vertices.shape[0],
                avg_sat=np.mean(cn_exists),
                avg_pred=np.mean(np.round(predictions))
            ),
            flush=True
        )
    # end
    return loss, acc, np.mean(cn_exists), np.mean(predictions), TP, FP, TN, FN


# end


def run_test_batch(sess, model, batch, batch_i, time_steps, logfile, runtabu=True):
    M, n_colors, VC, cn_exists, n_vertices, n_edges, f = batch

    # Compute the number of problems
    n_problems = n_vertices.shape[0]

    # open up the batch, which contains 2 instances
    for i in range(n_problems):
        n, m, c = n_vertices[i], n_edges[i], n_colors[i]
        conn = m / n
        n_acc = sum(n_vertices[0:i])
        c_acc = sum(n_colors[0:i])

        # subset adjacency matrix
        M_t = M[n_acc:n_acc + n, n_acc:n_acc + n]
        c = c if i % 2 == 0 else c + 1

        gnnpred = tabupred = 999
        for j in range(2, c + 5):
            n_colors_t = j
            cn_exists_t = 1 if n_colors_t >= c else 0
            VC_t = np.ones((n, n_colors_t))
            # Generate colors embeddings
            colors_initial_embeddings = np.random.rand(n_colors_t, d)

            feed_dict = {
                model['M']: M_t,
                model['VC']: VC_t,
                model['chrom_number']: np.array([n_colors_t]),
                model['time_steps']: time_steps,
                model['cn_exists']: np.array([cn_exists_t]),
                model['n_vertices']: np.array([n]),
                model['n_edges']: np.array([m]),
                model['colors_initial_embeddings']: colors_initial_embeddings
            }

            outputs = [model['loss'], model['acc'], model['predictions'], model['TP'], model['FP'], model['TN'],
                       model['FN']]

            # Run model - chromatic number or more
            init_time = timeit.default_timer()
            loss, acc, predictions, TP, FP, TN, FN = sess.run(outputs, feed_dict=feed_dict)[-7:]
            elapsed_gnn_time = timeit.default_timer() - init_time
            gnnpred = n_colors_t if predictions > 0.5 and n_colors_t < gnnpred else gnnpred

            # run tabucol
            if runtabu:
                init_time = timeit.default_timer()
                tabu_solution = tabucol(M_t, n_colors_t, max_iterations=1000)
                elapsed_tabu_time = timeit.default_timer() - init_time
                tabu_sol = 0 if tabu_solution is None else 1
                tabupred = n_colors_t if tabu_sol == 1 and n_colors_t < tabupred else tabupred
        # end for
        logfile.write(
            '{batch_i} {i} {n} {m} {conn} {tstloss} {tstacc} {cn_exists} {c} {gnnpred} {prediction} {gnntime} {'\
            'tabupred} {tabutime}\n'.format(
                batch_i=batch_i,
                i=i,
                n=n,
                m=m,
                c=c,
                conn=conn,
                cn_exists=cn_exists_t,
                tstloss=loss,
                tstacc=acc,
                gnnpred=gnnpred,
                prediction=predictions.item(),
                gnntime=elapsed_gnn_time,
                tabupred=tabupred if runtabu else 0,
                tabutime=elapsed_tabu_time if runtabu else 0
            )
        )
        logfile.flush()
    # end for batch


# end

def summarize_epoch(epoch_i, loss, acc, sat, pred, train=False):
    print(
        '{train_or_test} Epoch {epoch_i} Average\t|\t(Loss,Acc)=({loss:.4f},{acc:.4f})\t|\tAvg. (Sat,'
        'Pred)=({avg_sat:.4f},{avg_pred:.4f})'.format(
            train_or_test='Train' if train else 'Test',
            epoch_i=epoch_i,
            loss=np.mean(loss),
            acc=np.mean(acc),
            avg_sat=np.mean(sat),
            avg_pred=np.mean(pred)
        ),
        flush=True
    )


# end

# Set RNG seed for Python, Numpy and Tensorflow
random.seed(42)
np.random.seed(42)
tf.compat.v1.set_random_seed(42)
seed = str(42)
# Setup parameters
d = 64
time_steps = 32
epochs_n = 10000
batch_size = 8

loadpath = '.'
load_checkpoints = False
save_checkpoints = True

train_params = {
'batches_per_epoch': 128
}

test_params = {
'batches_per_epoch': 1
}

# Build model
print('Building model ...', flush=True)
tf.compat.v1.reset_default_graph()
GNN = build_network(d)

# Comment the following line to allow GPU use
config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True

with tf.Session(config=config) as sess:
    # Initialize global variables
    print('Initializing global variables ... ', flush=True)
    sess.run(tf.global_variables_initializer())

# Restore saved weights
if load_checkpoints: load_weights(sess, loadpath);


Building model ...


TypeError: __call__() missing 1 required positional argument: 'shape'

In [ ]:
# run training

train_loader = InstanceLoader('adversarial-training')

ptrain = 'training_' + seed
if not os.path.isdir(ptrain):
    os.makedirs(ptrain)
with open(ptrain + '/log.dat', 'w') as logfile:
    # Run for a number of epochs
    for epoch_i in np.arange(epochs_n):

        train_loader.reset()

        train_stats = {k: np.zeros(train_params['batches_per_epoch']) for k in
                       ['loss', 'acc', 'sat', 'pred', 'TP', 'FP', 'TN', 'FN']}

        print('Training model...', flush=True)
        for (batch_i, batch) in islice(enumerate(train_loader.get_batches(batch_size)),
                                       train_params['batches_per_epoch']):
            train_stats['loss'][batch_i], train_stats['acc'][batch_i], train_stats['sat'][batch_i], \
            train_stats['pred'][batch_i], train_stats['TP'][batch_i], train_stats['FP'][batch_i], \
            train_stats['TN'][batch_i], train_stats['FN'][batch_i] = run_training_batch(sess, GNN, batch,
                                                                                        batch_i, epoch_i,
                                                                                        time_steps, d,
                                                                                        verbose=True)
        # end
        summarize_epoch(epoch_i, train_stats['loss'], train_stats['acc'], train_stats['sat'],
                        train_stats['pred'], train=True)

        # Save weights
        savepath = ptrain + '/checkpoints/epoch={epoch}'.format(
            epoch=round(200 * np.ceil((epoch_i + 1) / 200)))
        os.makedirs(savepath, exist_ok=True)
        if save_checkpoints: save_weights(sess, savepath);

        logfile.write('{epoch_i} {trloss} {tracc} {trsat} {trpred} {trTP} {trFP} {trTN} {trFN} \n'.format(

            epoch_i=epoch_i,

            trloss=np.mean(train_stats['loss']),
            tracc=np.mean(train_stats['acc']),
            trsat=np.mean(train_stats['sat']),
            trpred=np.mean(train_stats['pred']),
            trTP=np.mean(train_stats['TP']),
            trFP=np.mean(train_stats['FP']),
            trTN=np.mean(train_stats['TN']),
            trFN=np.mean(train_stats['FN']),

        )
        )
        logfile.flush()
    # end
    # end


In [ ]:
# run testing

test_loader = InstanceLoader('adversarial-testing')
    
if not os.path.isdir('testing_' + seed):
    os.makedirs('testing_' + seed)
with open('testing_' + seed + '/log.dat', 'w') as logfile:

    test_loader.reset()
    logfile.write(
        'batch instance vertices edges connectivity loss acc sat chrom_number gnnpred gnncertainty '
        'gnntime tabupred tabutime\n')
    print('Testing model...', flush=True)
    for (batch_i, batch) in enumerate(test_loader.get_test_batches(1, 2048)):
        run_test_batch(sess, GNN, batch, batch_i, time_steps, logfile, runtabu)
    # end
    logfile.flush()
